In this tutorial, we build a simple matrix factorization model using the [MovieLens 100K dataset](https://grouplens.org/datasets/movielens/100k/) with TFRS. We can use this model to recommend movies for a given user.

In [1]:
import boto3
import sagemaker
import pandas as pd

sess   = sagemaker.Session()
bucket = sess.default_bucket()
role = sagemaker.get_execution_role()
region = boto3.Session().region_name

sm = boto3.Session().client(service_name='sagemaker', region_name=region)

# Create the Experiment

In [2]:
import time
from smexperiments.experiment import Experiment

timestamp = int(time.time())

recommender_experiment = Experiment.create(
                         experiment_name='MovieLens-Recommender-{}'.format(timestamp),
                         description='MovieLens Recommender', 
                         sagemaker_boto_client=sm)

recommender_experiment_name = recommender_experiment.experiment_name
print('Experiment name: {}'.format(recommender_experiment_name))

Experiment name: MovieLens-Recommender-1604258397


# Specify Input Data S3 URI and `Distribution Strategy`

In [3]:
from sagemaker.inputs import TrainingInput

input_train_data_s3_uri ='s3://sagemaker-us-east-1-835319576252/tensorflow_datasets/train/'

s3_input_train_data = TrainingInput(s3_data=input_train_data_s3_uri,
                                    distribution='ShardedByS3Key')
print(s3_input_train_data.config)

{'DataSource': {'S3DataSource': {'S3DataType': 'S3Prefix', 'S3Uri': 's3://sagemaker-us-east-1-835319576252/tensorflow_datasets/train/', 'S3DataDistributionType': 'ShardedByS3Key'}}}


# Setup Metrics To Track Model Performance

These sample log lines...
```
500/1000 [=====>..] - ETA: 3s - factorized_top_k/top_1_categorical_accuracy: 0.012 - factorized_top_k/top_5_categorical_accuracy: 0.225 - factorized_top_k/top_10_categorical_accuracy: 0.481 - factorized_top_k/top_50_categorical_accuracy: 0.607 - factorized_top_k/top_100_categorical_accuracy: 0.885
```
...will produce the following metrics in CloudWatch:

`factorized_top_k/top_1_categorical_accuracy` = 0.012

`factorized_top_k/top_5_categorical_accuracy` = 0.225

`factorized_top_k/top_10_categorical_accuracy` = 0.481

`factorized_top_k/top_50_categorical_accuracy` = 0.607

`factorized_top_k/top_100_categorical_accuracy` = 0.885

In [4]:
metrics_definitions = [
     {'Name': 'top_1_categorical_accuracy', 'Regex': 'factorized_top_k/top_1_categorical_accuracy: ([0-9\\.]+)'},
     {'Name': 'top_5_categorical_accuracy', 'Regex': 'factorized_top_k/top_5_categorical_accuracy: ([0-9\\.]+)'},
     {'Name': 'top_10_categorical_accuracy', 'Regex': 'factorized_top_k/top_10_categorical_accuracy: ([0-9\\.]+)'},
     {'Name': 'top_50_categorical_accuracy', 'Regex': 'factorized_top_k/top_50_categorical_accuracy: ([0-9\\.]+)'},
     {'Name': 'top_100_categorical_accuracy', 'Regex': 'factorized_top_k/top_100_categorical_accuracy: ([0-9\\.]+)'}
]

# Setup Hyper-Parameters for Classification Layer

In [5]:
epochs=100
learning_rate=0.5
dataset_variant='100k' # movielens 100k, 1m, 20m, 25m, etc
embedding_dimension=1024 # dimension (k) of our user and item embeddings
enable_tensorboard=True
train_instance_count=1
train_instance_type='ml.p3.2xlarge'

# Setup Our BERT + TensorFlow Script to Run on SageMaker
Prepare our TensorFlow model to run on the managed SageMaker service

In [6]:
!pygmentize src/train.py

import time
import random
import pandas as pd
from glob import glob
import pprint
import argparse
import json
import subprocess
import sys
import os

subprocess.check_call([sys.executable, '-m', 'pip', 'install', 'scikit-learn==0.23.1'])
subprocess.check_call([sys.executable, '-m', 'pip', 'install', 'sagemaker-tensorflow==2.3.0.1.0.0'])
subprocess.check_call([sys.executable, '-m', 'pip', 'install', 'tensorflow-recommenders==0.2.0'])
subprocess.check_call([sys.executable, '-m', 'pip', 'install', 'tensorflow-datasets==4.0.0'])
subprocess.check_call([sys.executable, '-m', 'pip', 'install', 'matplotlib==3.2.1'])
        
from typing import Dict, Text
import tensorflow as tf
import tensorflow_datasets as tfds
import tensorflow_recommenders as tfrs
import numpy as np

class MovieLensModel(tfrs.Model):
  # We derive from a custom base class to help reduce boilerplate. Under the hood,
  # these are still plain Keras Models.

  def __init__(
      self,
      user_embedding: tf.keras.Model,
   

In [7]:
from sagemaker.tensorflow import TensorFlow

estimator = TensorFlow(entry_point='train.py',
                       source_dir='src',
                       role=role,
                       instance_count=train_instance_count,
                       instance_type=train_instance_type,
                       py_version='py37',
                       framework_version='2.3.0',
                       hyperparameters={
                           'epochs': epochs,
                           'learning_rate': learning_rate,
                           'dataset_variant': dataset_variant,
                           'embedding_dimension': embedding_dimension,                           
                           'enable_tensorboard': enable_tensorboard
                       },
                       metric_definitions=metrics_definitions,
                       debugger_hook_config=False
            )

# Create the `Trial` and `Experiment Config`

In [18]:
import time
from smexperiments.trial import Trial

timestamp = int(time.time())

trial_name = 'trial-{}-{}-{}-{}'.format(timestamp, epochs, dataset_variant, embedding_dimension)

trial = Trial.create(trial_name=trial_name,
                     experiment_name=recommender_experiment_name,
                     sagemaker_boto_client=sm)

trial_name = trial.trial_name
print('Trial name: {}'.format(trial_name))

Trial name: trial-1604258571-100-100k-1024


In [21]:
recommender_experiment_config = {
    'ExperimentName': recommender_experiment_name,
    'TrialName': trial.trial_name,
    'TrialComponentDisplayName': 'train'
}

# Train the Model on SageMaker

In [22]:
estimator.fit(
              inputs={
                  'train': s3_input_train_data, 
              },              
              experiment_config=recommender_experiment_config,                   
              wait=False)

INFO:sagemaker:Creating training-job with name: tensorflow-training-2020-11-01-19-23-08-594


In [23]:
recommender_training_job_name = estimator.latest_training_job.name
print('Training Job Name:  {}'.format(recommender_training_job_name))

Training Job Name:  tensorflow-training-2020-11-01-19-23-08-594


In [24]:
from IPython.core.display import display, HTML

display(HTML('<b>Review <a target="blank" href="https://console.aws.amazon.com/sagemaker/home?region={}#/jobs/{}">Training Job</a></b>'.format(region, recommender_training_job_name)))


In [25]:
from IPython.core.display import display, HTML

display(HTML('<b>Review <a target="blank" href="https://console.aws.amazon.com/cloudwatch/home?region={}#logStream:group=/aws/sagemaker/TrainingJobs;prefix={};streamFilter=typeLogStreamPrefix">CloudWatch Logs</a></b>'.format(region, recommender_training_job_name)))


In [26]:
from IPython.core.display import display, HTML

display(HTML('<b>Review <a target="blank" href="https://s3.console.aws.amazon.com/s3/buckets/{}/{}/?region={}&tab=overview">S3 Output Data</a> After The Training Job Has Completed</b>'.format(bucket, recommender_training_job_name, region)))


# Wait for Training Job to Finish

In [ ]:
%%time

estimator.latest_training_job.wait(logs=False)


2020-11-01 19:23:10 Starting - Starting the training job
2020-11-01 19:23:12 Starting - Launching requested ML instances.............
2020-11-01 19:24:23 Starting - Preparing the instances for training............
2020-11-01 19:25:29 Downloading - Downloading input data.............
2020-11-01 19:26:41 Training - Downloading the training image....

# Copy the Trained Model from S3

In [ ]:
!aws s3 cp s3://$bucket/$recommender_training_job_name/output/model.tar.gz ./model.tar.gz

In [ ]:
!mkdir -p ./model/
!tar -xvzf ./model.tar.gz -C ./model/

# Inspect the Model

In [ ]:
!saved_model_cli show --all --dir ./model/tensorflow/saved_model/0/

# Make a Sample Prediction

In [ ]:
user_id="42"

In [ ]:
!saved_model_cli run --input_exprs 'input_1=np.array(["$user_id"])' --tag_set serve --signature_def serving_default --dir ./model/tensorflow/saved_model/0

# Show the Experiment Tracking Lineage

In [ ]:
from sagemaker.analytics import ExperimentAnalytics

lineage_table = ExperimentAnalytics(
    sagemaker_session=sess,
    experiment_name=recommender_experiment_name,
    metric_names=['top_1_categorical_accuracy',
                  'top_5_categorical_accuracy',
                  'top_10_categorical_accuracy',
                  'top_50_categorical_accuracy',
                  'top_100_categorical_accuracy'],
    sort_by="CreationTime",
    sort_order="Ascending",
)

lineage_df = lineage_table.dataframe()
lineage_df.shape

In [ ]:
lineage_df.columns

In [ ]:
lineage_df

In [ ]:
sm.describe_trial_component(TrialComponentName=lineage_df.TrialComponentName[0])

# Pass Variables to the Next Notebook(s)

In [ ]:
%store recommender_training_job_name

In [ ]:
# %%javascript
# Jupyter.notebook.save_checkpoint();
# Jupyter.notebook.session.delete();